<a href="https://colab.research.google.com/github/kweenkeen/ReachOutandRead/blob/main/Mail_Merge_Monthly_Email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
from datetime import date
from datetime import datetime

# Make sure to upload site data report for TWO REPORTING PERIODS!

site_data_report = files.upload()
site_data_file = next(iter(site_data_report))
df = pd.read_excel(site_data_file, converters={'Site ID': str})
df = df.set_index('Site ID')

Saving MyROR Site Data Report 2022-07-15.xlsx to MyROR Site Data Report 2022-07-15.xlsx


In [2]:
# Set filter
active_or_approved = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze', 'Approved - Needs Training', 'Approved - Waitlisted', 'In Progress'] 
df = df[df['Status'].isin(active_or_approved)] 

In [3]:
# UPLOAD AFFILIATE CONTACT LIST

affiliate_contact_list = files.upload()

Saving 2022.07.12 Find a Site Affiliate Contact Information.xlsx to 2022.07.12 Find a Site Affiliate Contact Information.xlsx


In [4]:
affiliate_filename = next(iter(affiliate_contact_list))

affiliate_contact_df = pd.read_excel(affiliate_filename)

df = df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')

In [5]:
df.head()

,Site Type,Site Name,Status,Legacy Satellite ID,Affiliate Name,Health System Name,Regions,Categories,Inactivated Reason,Physical Location: Line 1,...,AAB Date,Scholastic Account,Scholastic Balance,Scholastic Date,Most Recent Quality Classification Date,Most Recent Quality Classification Assessment,Most Recent Quality Classification Activity,"Site is encouraging parents/caregivers to talk, read, sing, and play with their babies at the WCV’s 0-5 months as instructed by the ROR anticipatory guidance",Site wants to give books at all or some WCVs from 0-5 months,Affiliate Contact
Site ID,,,,,,,,,,,,,,,,,,,,,
2,Primary,UAB Primary Care Clinic /,Active,NaN,Reach Out and Read Alabama,NaN,"Prescription for Summer Reading 2018, ACGLR - ...",NaN,NaN,1600 7th Avenue South,...,2022-07-14,2660370,2227.5,2022-07-11,2018-12-12,Green,2018-12-12,Y,Y,stsowell@roralabama.org
8,Primary,Quality of Life Health Services /,Active,NaN,Reach Out and Read Alabama,NaN,NaN,NaN,NaN,1411 Piedmont Cut-off,...,2022-07-14,4577849,0.0,2022-07-11,2018-06-01,Yellow,2018-06-01,Y,Y,stsowell@roralabama.org
24,Primary,Alaska Native Medical Center / PCC-Pediatric C...,Slated for deactivation,NaN,Independent / International,NaN,NaN,"ROR American Indian/Alaska Native, Grant Funde...",NaN,4320 Diplomacy Drive,...,NaT,7244523,0.0,2022-07-11,2017-08-02,Yellow,2017-08-02,NaN,NaN,NONE
29,Primary,Valleywise Community Health Center- Avondale /,Active,NaN,Reach Out and Read Arizona,NaN,"Southwest Human Development, AZ",NaN,NaN,950 E Van Buren Street,...,NaT,2693483,0.0,2022-07-11,2021-10-27,Yellow,2021-10-27,Y,Y,melanie@azaap.org
30,Primary,Valleywise Community Health Center- Chandler /,Active,NaN,Reach Out and Read Arizona,NaN,"Southwest Human Development, AZ",NaN,NaN,811 S Hamilton,...,NaT,4613947,0.0,2022-07-11,2021-10-22,Yellow,2021-10-22,Y,Y,melanie@azaap.org


In [6]:
rslt_df = df[['Affiliate Contact', 'Affiliate Name', 'Status', 'Join Date', 'Training %', 'Total Trained', 'Most Recent Quality Classification Date', 'Primary Contact Email', 'Medical Consultant Email']]

In [7]:
rslt_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date,Primary Contact Email,Medical Consultant Email
Site ID,,,,,,,,,
2,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1997-12-01,0.90,9.0,2018-12-12,jmckinney@peds.uab.edu,NaN
8,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1998-02-01,0.50,1.0,2018-06-01,tracie.williams@qolhs.com,deborah.smith@qolhs.com
24,NONE,Independent / International,Slated for deactivation,1997-04-01,0.67,8.0,2017-08-02,jbeaudion@southcentralfoundation.com,CTanCadogan@SouthcentralFoundation.com
29,melanie@azaap.org,Reach Out and Read Arizona,Active,2001-01-01,0.63,5.0,2021-10-27,Tatianna.Sanchez@valleywisehealth.org,Tina_McKenzie@dmgaz.org
30,melanie@azaap.org,Reach Out and Read Arizona,Active,2000-01-01,0.80,4.0,2021-10-22,Elizabeth.Menchaca@valleywisehealth.org,Laila_Sajwani@dmgaz.org


In [8]:
date_range_min = input("Type in the start of your year (MM/DD/YYYY): ")
date_range_min = datetime.strptime(date_range_min, '%m/%d/%Y')
date_range_max = input("Type in the end of your year (MM/DD/YYYY): ")
date_range_max = datetime.strptime(date_range_max, '%m/%d/%Y')

Type in the start of your year (MM/DD/YYYY): 07/01/2022
Type in the end of your year (MM/DD/YYYY): 07/15/2022


In [9]:
rslt_df['Join Date'] = pd.to_datetime(rslt_df['Join Date'], format='%m/%d/%Y')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
join_date_df = rslt_df.loc[(rslt_df['Join Date'] >= date_range_min) & (rslt_df['Join Date'] <= date_range_max)]
join_date_df['Join Date'] = join_date_df['Join Date'].dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
join_date_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date,Primary Contact Email,Medical Consultant Email
Site ID,,,,,,,,,
2439.6,jessie@reachoutandreadco.org,Reach Out and Read Colorado,Active,2022-07-11,1.00,1.0,NaT,SSantana@mountainfamily.org,aheckert@mountainfamily.org
2988.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,8.0,NaT,bsullivan@glpediatrics.com,jenbell39@gmail.com
3973.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,5.0,NaT,mfoley@hydeparkpedi.com,cwalsh@hydeparkpedi.com
7136.3,dharrisschaap@readyforschool.org,Michigan (Mars Pathfinder),Active,2022-07-13,1.00,2.0,NaT,csoule@mckenziehealth.org,hchambers@mckenziehealth.org
11989.2,Hannah.Slaughter@reachoutandread.org,Reach Out and Read Washington,Active,2022-07-05,0.83,5.0,NaT,NaN,NaN


In [12]:
join_date_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date,Primary Contact Email,Medical Consultant Email
Site ID,,,,,,,,,
2439.6,jessie@reachoutandreadco.org,Reach Out and Read Colorado,Active,2022-07-11,1.00,1.0,NaT,SSantana@mountainfamily.org,aheckert@mountainfamily.org
2988.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,8.0,NaT,bsullivan@glpediatrics.com,jenbell39@gmail.com
3973.2,Meghan.Blomberg@reachoutandread.org,Reach Out and Read Massachusetts,Active,2022-07-11,1.00,5.0,NaT,mfoley@hydeparkpedi.com,cwalsh@hydeparkpedi.com
7136.3,dharrisschaap@readyforschool.org,Michigan (Mars Pathfinder),Active,2022-07-13,1.00,2.0,NaT,csoule@mckenziehealth.org,hchambers@mckenziehealth.org
11989.2,Hannah.Slaughter@reachoutandread.org,Reach Out and Read Washington,Active,2022-07-05,0.83,5.0,NaT,NaN,NaN


In [13]:
rslt_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date,Primary Contact Email,Medical Consultant Email
Site ID,,,,,,,,,
2,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1997-12-01,0.90,9.0,2018-12-12,jmckinney@peds.uab.edu,NaN
8,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1998-02-01,0.50,1.0,2018-06-01,tracie.williams@qolhs.com,deborah.smith@qolhs.com
24,NONE,Independent / International,Slated for deactivation,1997-04-01,0.67,8.0,2017-08-02,jbeaudion@southcentralfoundation.com,CTanCadogan@SouthcentralFoundation.com
29,melanie@azaap.org,Reach Out and Read Arizona,Active,2001-01-01,0.63,5.0,2021-10-27,Tatianna.Sanchez@valleywisehealth.org,Tina_McKenzie@dmgaz.org
30,melanie@azaap.org,Reach Out and Read Arizona,Active,2000-01-01,0.80,4.0,2021-10-22,Elizabeth.Menchaca@valleywisehealth.org,Laila_Sajwani@dmgaz.org


In [14]:
mail_merge_df = pd.DataFrame(columns=['Affiliate Name'])

In [15]:
mail_merge_df = mail_merge_df.set_index('Affiliate Name')

In [16]:
mail_merge_df.head()

""
Affiliate Name


In [17]:
# join_date_df = join_date_df['Affiliate Name'].value_counts()

In [18]:
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
active_df = rslt_df[rslt_df['Status'].isin(high_level_active)] 

In [19]:
active_sites_df = active_df['Affiliate Name'].value_counts()

In [20]:
active_sites_df =active_sites_df.transpose()

In [21]:
active_sites_df.head()

Independent / International          656
Reach Out and Read North Carolina    375
Reach Out and Read Colorado          329
Reach Out and Read Minnesota         301
Reach Out and Read Massachusetts     268
Name: Affiliate Name, dtype: int64

In [22]:
mail_merge_df['Number of Sites (High Level Active)'] = active_sites_df

In [23]:
mail_merge_df.head()

,Number of Sites (High Level Active)
Affiliate Name,
Independent / International,656
Reach Out and Read North Carolina,375
Reach Out and Read Colorado,329
Reach Out and Read Minnesota,301
Reach Out and Read Massachusetts,268


In [24]:
new_sites_df = join_date_df['Affiliate Name'].value_counts()

In [25]:
mail_merge_df['Number of New Sites'] = new_sites_df

In [26]:
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].fillna(0)

In [27]:
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].astype(int)

In [28]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites
Affiliate Name,,
Independent / International,656,1
Reach Out and Read North Carolina,375,0
Reach Out and Read Colorado,329,1
Reach Out and Read Minnesota,301,1
Reach Out and Read Massachusetts,268,2


In [29]:
active_df.head()

,Affiliate Contact,Affiliate Name,Status,Join Date,Training %,Total Trained,Most Recent Quality Classification Date,Primary Contact Email,Medical Consultant Email
Site ID,,,,,,,,,
2,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1997-12-01,0.90,9.0,2018-12-12,jmckinney@peds.uab.edu,NaN
8,stsowell@roralabama.org,Reach Out and Read Alabama,Active,1998-02-01,0.50,1.0,2018-06-01,tracie.williams@qolhs.com,deborah.smith@qolhs.com
24,NONE,Independent / International,Slated for deactivation,1997-04-01,0.67,8.0,2017-08-02,jbeaudion@southcentralfoundation.com,CTanCadogan@SouthcentralFoundation.com
29,melanie@azaap.org,Reach Out and Read Arizona,Active,2001-01-01,0.63,5.0,2021-10-27,Tatianna.Sanchez@valleywisehealth.org,Tina_McKenzie@dmgaz.org
30,melanie@azaap.org,Reach Out and Read Arizona,Active,2000-01-01,0.80,4.0,2021-10-22,Elizabeth.Menchaca@valleywisehealth.org,Laila_Sajwani@dmgaz.org


In [30]:
training_percent_average_df = active_df.groupby('Affiliate Name')['Training %'].mean()

In [31]:
training_percent_average_df.head()

Affiliate Name
Delaware (TIROS 1)                     0.220000
Idaho (Curiosity Rover)                0.760909
Independent / International            0.686371
Michigan (Mars Pathfinder)             0.788969
PA (excluding Greater Philadelphia)    0.826892
Name: Training %, dtype: float64

In [32]:
mail_merge_df['Current Overall Training Percentage'] = training_percent_average_df

In [33]:
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage']* 100

In [34]:
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(int)

In [35]:
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(str) + '%'

In [36]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage
Affiliate Name,,,
Independent / International,656,1,68%
Reach Out and Read North Carolina,375,0,93%
Reach Out and Read Colorado,329,1,92%
Reach Out and Read Minnesota,301,1,86%
Reach Out and Read Massachusetts,268,2,98%


In [37]:
providers_trained_count_df = active_df.groupby('Affiliate Name')['Total Trained'].sum().astype(int)

In [38]:
providers_trained_count_df.head()

Affiliate Name
Delaware (TIROS 1)                       17
Idaho (Curiosity Rover)                  95
Independent / International            2333
Michigan (Mars Pathfinder)              339
PA (excluding Greater Philadelphia)    1164
Name: Total Trained, dtype: int64

In [39]:
mail_merge_df['Number of Providers Trained'] = providers_trained_count_df

In [40]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained
Affiliate Name,,,,
Independent / International,656,1,68%,2333
Reach Out and Read North Carolina,375,0,93%,2001
Reach Out and Read Colorado,329,1,92%,1635
Reach Out and Read Minnesota,301,1,86%,2385
Reach Out and Read Massachusetts,268,2,98%,1823


In [41]:
SQC_date_df = rslt_df.loc[(rslt_df['Most Recent Quality Classification Date'] >= date_range_min) & (rslt_df['Most Recent Quality Classification Date'] <= date_range_max)]
SQC_date_df['Most Recent Quality Classification Date'] = SQC_date_df['Most Recent Quality Classification Date'].dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
SQC_updates_df = SQC_date_df['Affiliate Name'].value_counts()

In [43]:
SQC_updates_df.head()

Reach Out and Read Texas                         11
Reach Out and Read San Diego (California)         7
Reach Out and Read Georgia                        7
Reach Out and Read Orange County (California)     7
Reach Out and Read Washington                     6
Name: Affiliate Name, dtype: int64

In [44]:
mail_merge_df['# of SQCs Completed'] =SQC_updates_df
mail_merge_df['# of SQCs Completed'] = mail_merge_df['# of SQCs Completed'].fillna(0)
mail_merge_df['# of SQCs Completed'] = mail_merge_df['# of SQCs Completed'].astype(int)

In [45]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed
Affiliate Name,,,,,
Independent / International,656,1,68%,2333,0
Reach Out and Read North Carolina,375,0,93%,2001,0
Reach Out and Read Colorado,329,1,92%,1635,0
Reach Out and Read Minnesota,301,1,86%,2385,0
Reach Out and Read Massachusetts,268,2,98%,1823,5


In [46]:
in_progress_df = df[df['Status'] == 'In Progress'] 

In [47]:
in_progress_df = in_progress_df['Affiliate Name'].value_counts()

In [48]:
in_progress_df = in_progress_df.transpose()

In [49]:
in_progress_df.head()

Reach Out and Read Georgia       120
Independent / International       95
Reach Out and Read Illinois       76
Reach Out and Read Wisconsin      67
Reach Out and Read Washington     52
Name: Affiliate Name, dtype: int64

In [50]:
mail_merge_df['# of Applications in Progress'] = in_progress_df

In [51]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress
Affiliate Name,,,,,,
Independent / International,656,1,68%,2333,0,95
Reach Out and Read North Carolina,375,0,93%,2001,0,40
Reach Out and Read Colorado,329,1,92%,1635,0,13
Reach Out and Read Minnesota,301,1,86%,2385,0,17
Reach Out and Read Massachusetts,268,2,98%,1823,5,17


In [52]:
approved_needs_training_df = df[df['Status'] == 'Approved - Needs Training'] 
approved_needs_training_df = approved_needs_training_df['Affiliate Name'].value_counts()
approved_needs_training_df = approved_needs_training_df.transpose()
mail_merge_df['# of Approved - Needs Training Sites'] = approved_needs_training_df
mail_merge_df['# of Approved - Needs Training Sites'] = mail_merge_df['# of Approved - Needs Training Sites'].fillna(0)
mail_merge_df['# of Approved - Needs Training Sites'] = mail_merge_df['# of Approved - Needs Training Sites'].astype(int)

In [53]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites
Affiliate Name,,,,,,,
Independent / International,656,1,68%,2333,0,95,56
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13
Reach Out and Read Colorado,329,1,92%,1635,0,13,4
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3


In [54]:
approved_waitlisted_df = df[df['Status'] == 'Approved - Waitlisted'] 
approved_waitlisted_df = approved_waitlisted_df['Affiliate Name'].value_counts()
approved_waitlisted_df = approved_waitlisted_df.transpose()
mail_merge_df['# of Approved - Waitlisted Sites'] = approved_waitlisted_df
mail_merge_df['# of Approved - Waitlisted Sites'] = mail_merge_df['# of Approved - Waitlisted Sites'].fillna(0)
mail_merge_df['# of Approved - Waitlisted Sites'] = mail_merge_df['# of Approved - Waitlisted Sites'].astype(int)

In [55]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites
Affiliate Name,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3


In [56]:
covid_freeze_df = df[df['Status'] == 'COVID-19 Freeze'] 
covid_freeze_df = covid_freeze_df['Affiliate Name'].value_counts()
covid_freeze_df = covid_freeze_df.transpose()
mail_merge_df['# of Covid Freeze Sites'] = covid_freeze_df
mail_merge_df['# of Covid Freeze Sites'] = mail_merge_df['# of Covid Freeze Sites'].fillna(0)
mail_merge_df['# of Covid Freeze Sites'] = mail_merge_df['# of Covid Freeze Sites'].astype(int)

In [57]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites
Affiliate Name,,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9,4
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3,2
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0,1
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3,3


In [58]:
S4D_df = df[df['Status'] == 'Slated for deactivation'] 
S4D_df = S4D_df['Affiliate Name'].value_counts()
S4D_df = S4D_df.transpose()
mail_merge_df['# of Slated for Deactivation Sites'] = S4D_df
mail_merge_df['# of Slated for Deactivation Sites'] = mail_merge_df['# of Slated for Deactivation Sites'].fillna(0)
mail_merge_df['# of Slated for Deactivation Sites'] = mail_merge_df['# of Slated for Deactivation Sites'].astype(int)

In [59]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites
Affiliate Name,,,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9,4,81
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3,2,0
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0,0
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0,1,0
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3,3,0


In [60]:
contact_list1 = []
contact_list2 = []
contact_list3 = []
affiliate_list1 = []
affiliate_list2 = []
affiliate_list3 = []

for ind, row in rslt_df.iterrows():
  email_check1 = rslt_df['Primary Contact Email'][ind]
  email_check2 = rslt_df['Medical Consultant Email'][ind]
  email_check3 = rslt_df['Affiliate Contact'][ind]

  # If there is NO  Primary Contact Email, substitue Medical Consultant Email for Primary Contact Email
  if type(email_check1) == float:
    contact_list1.append(ind)
    affiliate_list1.append(rslt_df.loc[ind, "Affiliate Name"])
  
  if type(email_check2) == float:
    contact_list2.append(ind)
    affiliate_list2.append(rslt_df.loc[ind, "Affiliate Name"])

  # If there is NO Primary Contcat Email AND NO medical contact email, substitute info@reachoutandread.org
  elif type(email_check1) == float and type(email_check2) == float:
    contact_list3.append(ind)
    affiliate_list3.append(rslt_df.loc[ind, "Affiliate Name"])

contact_df1 = pd.DataFrame({'Missing Primary Contact': pd.Series(contact_list1), 'Affiliate Name': pd.Series(affiliate_list1)})
contact_df2 = pd.DataFrame({'Missing Medical Contact': pd.Series(contact_list2), 'Affiliate Name': pd.Series(affiliate_list2)})
contact_df3 = pd.DataFrame({'Missing Primary Contact and Medical Contact': pd.Series(contact_list3), 'Affiliate Name': pd.Series(affiliate_list3)})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [61]:
contact_df1.head()

,Missing Primary Contact,Affiliate Name
0,40.3,Reach Out and Read Arizona
1,70.2,Reach Out and Read Bay Area
2,92,Reach Out and Read San Diego (California)
3,287.3,Reach Out and Read Illinois
4,287.4,Reach Out and Read Illinois


In [62]:
missing_primary_contact_df = contact_df1['Affiliate Name'].value_counts()
missing_primary_contact_df = missing_primary_contact_df.transpose()
mail_merge_df['# of Missing Primary Contacts'] = missing_primary_contact_df
mail_merge_df['# of Missing Primary Contacts'] = mail_merge_df['# of Missing Primary Contacts'].fillna(0)
mail_merge_df['# of Missing Primary Contacts'] = mail_merge_df['# of Missing Primary Contacts'].astype(int)

In [63]:
missing_primary_contact_df.head()

Reach Out and Read Illinois    29
Independent / International    25
Reach Out and Read Georgia     23
Reach Out and Read Iowa        19
Reach Out and Read Indiana     19
Name: Affiliate Name, dtype: int64

In [64]:
missing_medical_contact_df = contact_df2['Affiliate Name'].value_counts()
missing_medical_contact_df = missing_medical_contact_df.transpose()
mail_merge_df['# of Missing Medical Consultants'] = missing_medical_contact_df
mail_merge_df['# of Missing Medical Consultants'] = mail_merge_df['# of Missing Medical Consultants'].fillna(0)
mail_merge_df['# of Missing Medical Consultants'] = mail_merge_df['# of Missing Medical Consultants'].astype(int)

In [65]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites,# of Missing Primary Contacts,# of Missing Medical Consultants
Affiliate Name,,,,,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9,4,81,25,284
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3,2,0,10,41
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0,0,13,103
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0,1,0,11,47
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3,3,0,6,22


In [66]:
mail_merge_df = mail_merge_df.sort_values(by='Affiliate Name')

In [67]:
mail_merge_df

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites,# of Missing Primary Contacts,# of Missing Medical Consultants
Affiliate Name,,,,,,,,,,,,
Delaware (TIROS 1),16,0,22%,17,0,4,2,1,0,0,0,2
Idaho (Curiosity Rover),22,0,76%,95,0,3,7,0,0,1,1,6
Independent / International,656,1,68%,2333,0,95,56,9,4,81,25,284
Michigan (Mars Pathfinder),120,2,78%,339,0,12,9,4,0,11,6,57
PA (excluding Greater Philadelphia),165,0,82%,1164,0,14,11,0,0,4,13,48
Reach Out and Read Alabama,59,0,79%,218,0,13,7,14,2,3,16,28
Reach Out and Read Arizona,186,0,82%,607,1,39,3,1,7,2,14,44
Reach Out and Read Bay Area,166,0,51%,368,3,11,3,0,0,24,15,63
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0,0,13,103


In [68]:
mail_merge_df = mail_merge_df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')

In [69]:
mail_merge_df

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites,# of Missing Primary Contacts,# of Missing Medical Consultants,Affiliate Contact
Affiliate Name,,,,,,,,,,,,,
Delaware (TIROS 1),16,0,22%,17,0,4,2,1,0,0,0,2,alyssa.brunstein@reachoutandread.org
Idaho (Curiosity Rover),22,0,76%,95,0,3,7,0,0,1,1,6,Hannah.Slaughter@reachoutandread.org
Independent / International,656,1,68%,2333,0,95,56,9,4,81,25,284,NONE
Michigan (Mars Pathfinder),120,2,78%,339,0,12,9,4,0,11,6,57,dharrisschaap@readyforschool.org
PA (excluding Greater Philadelphia),165,0,82%,1164,0,14,11,0,0,4,13,48,NONE
Reach Out and Read Alabama,59,0,79%,218,0,13,7,14,2,3,16,28,stsowell@roralabama.org
Reach Out and Read Arizona,186,0,82%,607,1,39,3,1,7,2,14,44,melanie@azaap.org
Reach Out and Read Bay Area,166,0,51%,368,3,11,3,0,0,24,15,63,NONE
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0,0,13,103,jessie@reachoutandreadco.org


In [70]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites,# of Missing Primary Contacts,# of Missing Medical Consultants,Affiliate Contact
Affiliate Name,,,,,,,,,,,,,
Delaware (TIROS 1),16,0,22%,17,0,4,2,1,0,0,0,2,alyssa.brunstein@reachoutandread.org
Idaho (Curiosity Rover),22,0,76%,95,0,3,7,0,0,1,1,6,Hannah.Slaughter@reachoutandread.org
Independent / International,656,1,68%,2333,0,95,56,9,4,81,25,284,NONE
Michigan (Mars Pathfinder),120,2,78%,339,0,12,9,4,0,11,6,57,dharrisschaap@readyforschool.org
PA (excluding Greater Philadelphia),165,0,82%,1164,0,14,11,0,0,4,13,48,NONE


In [71]:
# SAVE RESULTS DATAFRAME AS EXCEL FILE 
today = str(date.today())
excel_filename = "Mail_Merge_" + today + ".xlsx"
mail_merge_df.to_excel(excel_filename)
files.download(excel_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>